<a href="https://colab.research.google.com/github/mjmaher987/Stock-Prediction-Using-ML/blob/main/Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NBEATS

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load your DataFrame
data = pd.read_csv('EURUSD_5min.csv')

# Preprocessing
scaler = MinMaxScaler()
data['close'] = scaler.fit_transform(data['close'].values.reshape(-1, 1))

# Splitting into training and testing sets
train_size = int(0.8 * len(data))
train_data, test_data = data[:train_size], data[train_size:]


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [3]:
# Prepare data for N-BEATS
window_size = 10
X, y = [], []
for i in range(len(data) - window_size):
    X.append(data['close'][i:i+window_size])
    y.append(data['close'][i+window_size])
X = np.array(X)
y = np.array(y)

train_size = int(0.8 * len(X))
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]

# Build N-BEATS model
def build_nbeats(input_shape, output_shape, num_blocks=4, num_neurons=128):
    inputs = Input(shape=input_shape)
    x = inputs
    forecasts = []
    for _ in range(num_blocks):
        for _ in range(4):  # Directly following the N-BEATS architecture
            x = Dense(num_neurons, activation='relu')(x)
        forecast = Dense(output_shape[0])(x)
        forecasts.append(forecast)
    outputs = forecasts[0]  # Use the forecast from the first block as output
    model = Model(inputs, outputs)
    return model

input_shape = (window_size, 1)
output_shape = (1,)
nbeats_model = build_nbeats(input_shape, output_shape)
nbeats_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train N-BEATS model
nbeats_model.fit(train_X, train_y, epochs=50, batch_size=32, verbose=1)

# Make predictions using the trained model
test_predictions = nbeats_model.predict(test_X)


Epoch 1/50
125/125 [==============================] - 5s 12ms/step - loss: 0.0854
Epoch 2/50
125/125 [==============================] - 2s 12ms/step - loss: 0.0590
Epoch 3/50
125/125 [==============================] - 2s 13ms/step - loss: 0.0593
Epoch 4/50
125/125 [==============================] - 1s 12ms/step - loss: 0.0595
Epoch 5/50
125/125 [==============================] - 2s 12ms/step - loss: 0.0586
Epoch 6/50
125/125 [==============================] - 2s 13ms/step - loss: 0.0591
Epoch 7/50
125/125 [==============================] - 2s 15ms/step - loss: 0.0583
Epoch 8/50
125/125 [==============================] - 2s 13ms/step - loss: 0.0583
Epoch 9/50
125/125 [==============================] - 2s 12ms/step - loss: 0.0586
Epoch 10/50
125/125 [==============================] - 1s 12ms/step - loss: 0.0589
Epoch 11/50
125/125 [==============================] - 1s 11ms/step - loss: 0.0586
Epoch 12/50
125/125 [==============================] - 1s 9ms/step - loss: 0.0584
Epoch 13/50
12

In [4]:
reshaped_test_predictions = test_predictions[:, -1, :]

In [5]:
# # Calculate MAE and RMSE
# print(test_y.shape)
# print(reshaped_test_predictions.shape)
mae = mean_absolute_error(test_y, reshaped_test_predictions)
rmse = mean_squared_error(test_y, reshaped_test_predictions, squared=False)
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 0.25914657814572256
Root Mean Squared Error: 0.2741693059599835


#NHEATS

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load and preprocess your data
# data = pd.read_csv('trading_data.csv')
# scaler = MinMaxScaler()
# data['close'] = scaler.fit_transform(data['close'].values.reshape(-1, 1))

# Prepare features and target variable
X = data[['open', 'high', 'low', 'volume']].values
y = data['close'].values

# Split data into train, validation, and test sets
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))
train_X, test_X, train_y, test_y = X[:train_size], X[train_size:], y[:train_size], y[train_size:]
train_X, val_X, train_y, val_y = train_X[:train_size - val_size], train_X[train_size - val_size:], \
                                  train_y[:train_size - val_size], train_y[train_size - val_size:]

# Normalize data
scaler_X = MinMaxScaler()
train_X = scaler_X.fit_transform(train_X)
val_X = scaler_X.transform(val_X)
test_X = scaler_X.transform(test_X)

scaler_y = MinMaxScaler()
train_y = scaler_y.fit_transform(train_y.reshape(-1, 1)).flatten()
val_y = scaler_y.transform(val_y.reshape(-1, 1)).flatten()
test_y = scaler_y.transform(test_y.reshape(-1, 1)).flatten()

# NHITS Model
def build_nhits(input_shape):
    model = tf.keras.Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    return model

nhits_model = build_nhits(train_X.shape[1])
nhits_model.compile(optimizer='adam', loss='mean_squared_error')

nhits_model.fit(train_X, train_y, epochs=50, batch_size=32, validation_data=(val_X, val_y), verbose=1)

nhits_predictions = nhits_model.predict(test_X)

# Calculate MAE and RMSE for NHITS
nhits_mae = mean_absolute_error(test_y, nhits_predictions)
nhits_rmse = mean_squared_error(test_y, nhits_predictions, squared=False)
print(f"NHITS Mean Absolute Error: {nhits_mae}")
print(f"NHITS Root Mean Squared Error: {nhits_rmse}")




Epoch 1/50
86/86 [==============================] - 1s 5ms/step - loss: 0.0515 - val_loss: 1.8923e-04
Epoch 2/50
86/86 [==============================] - 0s 3ms/step - loss: 4.6590e-05 - val_loss: 3.6330e-05
Epoch 3/50
86/86 [==============================] - 0s 3ms/step - loss: 2.9555e-05 - val_loss: 2.3878e-05
Epoch 4/50
86/86 [==============================] - 0s 3ms/step - loss: 2.6863e-05 - val_loss: 2.3510e-05
Epoch 5/50
86/86 [==============================] - 0s 3ms/step - loss: 2.4796e-05 - val_loss: 2.2684e-05
Epoch 6/50
86/86 [==============================] - 0s 2ms/step - loss: 2.4372e-05 - val_loss: 2.4931e-05
Epoch 7/50
86/86 [==============================] - 0s 2ms/step - loss: 2.4061e-05 - val_loss: 2.2347e-05
Epoch 8/50
86/86 [==============================] - 0s 3ms/step - loss: 2.4340e-05 - val_loss: 3.1715e-05
Epoch 9/50
86/86 [==============================] - 0s 2ms/step - loss: 2.4540e-05 - val_loss: 2.6735e-05
Epoch 10/50
86/86 [==============================]

#RNN

In [7]:
# RNN Model
def build_rnn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(input_shape, 1), activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    return model

train_X_rnn = train_X.reshape(train_X.shape[0], train_X.shape[1], 1)
val_X_rnn = val_X.reshape(val_X.shape[0], val_X.shape[1], 1)
test_X_rnn = test_X.reshape(test_X.shape[0], test_X.shape[1], 1)

rnn_model = build_rnn(train_X_rnn.shape[1])
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

rnn_model.fit(train_X_rnn, train_y, epochs=50, batch_size=32, validation_data=(val_X_rnn, val_y), verbose=1)


# Evaluate RNN models
rnn_predictions = rnn_model.predict(test_X_rnn)

# Calculate MAE and RMSE for RNN
rnn_mae = mean_absolute_error(test_y, rnn_predictions)
rnn_rmse = mean_squared_error(test_y, rnn_predictions, squared=False)
print(f"RNN Mean Absolute Error: {rnn_mae}")
print(f"RNN Root Mean Squared Error: {rnn_rmse}")


Epoch 1/50
86/86 [==============================] - 2s 8ms/step - loss: 0.1862 - val_loss: 0.0025
Epoch 2/50
86/86 [==============================] - 1s 9ms/step - loss: 0.0063 - val_loss: 0.0013
Epoch 3/50
86/86 [==============================] - 1s 16ms/step - loss: 4.0841e-04 - val_loss: 2.0742e-04
Epoch 4/50
86/86 [==============================] - 2s 18ms/step - loss: 1.6705e-04 - val_loss: 1.4413e-04
Epoch 5/50
86/86 [==============================] - 0s 5ms/step - loss: 1.4508e-04 - val_loss: 1.3523e-04
Epoch 6/50
86/86 [==============================] - 0s 5ms/step - loss: 1.3096e-04 - val_loss: 1.5602e-04
Epoch 7/50
86/86 [==============================] - 0s 5ms/step - loss: 1.2499e-04 - val_loss: 1.0099e-04
Epoch 8/50
86/86 [==============================] - 0s 5ms/step - loss: 1.1088e-04 - val_loss: 9.2638e-05
Epoch 9/50
86/86 [==============================] - 0s 5ms/step - loss: 1.0062e-04 - val_loss: 1.5743e-04
Epoch 10/50
86/86 [==============================] - 0s 6ms/